In [1]:
import data_utils as my_utils
import cv2
import dlib
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

import keras
from keras.callbacks import ModelCheckpoint
from keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, Input, MaxPool2D
from keras.models import Sequential, Model
from keras.utils import np_utils
from skimage.feature import hog
import sklearn
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [2]:
df = pd.read_csv("fer2013.csv")
emotion_labels = {0: "Angry", 1: "Disgust", 2:"Fear", 3:"Happy", 4:"Sad", 5:"Surprise", 6:"Neutral"}
labels = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]

In [3]:
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [4]:
df["Usage"].value_counts()

Training       28709
PublicTest      3589
PrivateTest     3589
Name: Usage, dtype: int64

In [5]:
df["emotion"].value_counts()

3    8989
6    6198
4    6077
2    5121
0    4953
5    4002
1     547
Name: emotion, dtype: int64

In [6]:
emotion_labels

{0: 'Angry',
 1: 'Disgust',
 2: 'Fear',
 3: 'Happy',
 4: 'Sad',
 5: 'Surprise',
 6: 'Neutral'}

In [7]:
# process the training, validation and test data
x_train, y_train = my_utils.process_data(df, 'Training')
x_validation, y_validation = my_utils.process_data(df, 'PublicTest')
x_test, y_test = my_utils.process_data(df, "PrivateTest")

/Users/ritvikareddy 1/Documents/GitHub/ml-project/venv/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/Users/ritvikareddy 1/Documents/GitHub/ml-project/data_utils.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  lambda pixel_str: np.fromstring(pixel_str, sep=' '))


In [11]:
# get face landmarks
x_train_landmarks = my_utils.extract_landmark_features(x_train)
x_validation_landmarks = my_utils.extract_landmark_features(x_validation)
x_test_landmarks = my_utils.extract_landmark_features(x_test)

In [ ]:
# cnn for just the raw pixels
images_nn = Input(shape=(48,48,1))

images_conv1 = Conv2D(64, 3, kernel_initializer='he_normal', activation='relu')(images_nn)
images_batchnorm1 = BatchNormalization()(images_conv1)
images_pool1 = MaxPool2D(pool_size=(3,3), strides=2)(images_batchnorm1)

images_conv2 = Conv2D(128, 3, activation='relu')(images_pool1)
images_batchnorm2 = BatchNormalization()(images_conv2)
images_pool2 = MaxPool2D(pool_size=(3,3), strides=2)(images_batchnorm2)

images_conv3 = Conv2D(256, 3, activation='relu')(images_pool2)
images_batchnorm3 = BatchNormalization()(images_conv3)
images_pool3 = MaxPool2D(pool_size=(3,3), strides=2)(images_batchnorm3)
images_dropout1 = Dropout(0.6)(images_pool3)

images_flat1 = Flatten()(images_dropout1)
images_dense1 = Dense(4096, activation='relu')(images_flat1)
images_dropout2 = Dropout(0.6)(images_dense1)

images_dense2 = Dense(1024, activation='relu')(images_dropout2)
images_batchnorm4 = BatchNormalization()(images_dense2)

images_dense3 = Dense(128, activation='relu')(images_batchnorm4)


# cnn for landmark features
# input shape will be 68*2 (from landamrks)
landmarks_features_nn = Input(shape=(68,2))
landmarks_features_flat1 = Flatten()(landmarks_features_nn)
landmarks_features_dense1 = Dense(1024, activation='relu')(landmarks_features_flat1)
landmarks_features_batchnorm1 = BatchNormalization()(landmarks_features_dense1)

landmarks_features_dense2 = Dense(128, activation='relu') (landmarks_features_batchnorm1)
landmarks_features_batchnorm2 = BatchNormalization()(landmarks_features_dense2)

# merge final outputs from both networks
merged_net = keras.layers.merge.concatenate([images_dense3, landmarks_features_batchnorm2],
                                           axis=-1)

output_layer = Dense(7, activation='softmax')(merged_net)

model = Model(inputs=[images_nn, landmarks_features_nn], outputs=output_layer)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
# save best weights
checkpoint = ModelCheckpoint(filepath='cnn_arch2_landmarks', verbose=2, save_best_only=True)

# num epochs
epochs = 10

# run model
model_history = model.fit(x_train, y_train, epochs=epochs,
                 shuffle=True,
                 batch_size=100, validation_data=(x_validation, y_validation),
                 callbacks=[checkpoint], verbose=2)

# save model to json
model_json = model.to_json()
with open("cnn_arch2_landmarks.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1, 2, 1)
plt.suptitle('CNN on raw pixel data', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(model_history.history['loss'], color='b', label='Training Loss')
plt.plot(model_history.history['val_loss'], color='r', label='Validation Loss')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(model_history.history['acc'], color='b', label='Training Accuracy')
plt.plot(model_history.history['val_acc'], color='r', label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()

In [ ]:
y_predicted_probs = model.predict(x_test, verbose=1)

In [ ]:
score = model.evaluate(x_test, y_test, verbose=1)

In [ ]:
print("Metrics: ", model.metrics_names)
print("Loss on test data", score[0])
print("Test Accuracy", score[1])

In [ ]:
def convert_probs_to_labels(y_probs, labels):
    predicted_label = lambda x: labels[x.argmax()]
    return [predicted_label(each) for each in y_probs]

In [ ]:
y_predicted_labels = convert_probs_to_labels(y_predicted_probs, labels)
y_actual_labels = convert_probs_to_labels(y_test, labels)

In [ ]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
cnf_matrix = confusion_matrix(y_actual_labels, y_predicted_labels)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
fig = plt.figure(figsize=(8,6))
plot_confusion_matrix(cnf_matrix, classes=labels,
                      title='Confusion Matrix for Test Data')
plt.show()
fig.savefig('confusion_matrix.png')

In [ ]:
def test_for_single_image(file_name, model, labels):
    image_pixels = cv2.imread(file_name, 0)
#     print(len(image_pixels))
    image_input = image_pixels.reshape(-1, 48, 48, 1)
    predicted_emotion = model.predict(image_input, verbose=0)
#     print(predicted_emotion)
    print("This image is classified to be {} with {:.2f}% confidence".format(
        labels[predicted_emotion.argmax()],predicted_emotion.max()*100))
    

In [ ]:
test_for_single_image('100th.jpg', model, labels)

In [ ]:
test_for_single_image('ashok.jpg', model, labels)

In [ ]:
test_for_single_image('ashok2.png', model, labels)

In [ ]:
test_for_single_image('rose2.jpg', model, labels)

In [ ]:
test_for_single_image('rose3.jpg', model, labels)

In [ ]:
test_for_single_image('jose.png', model, labels)

In [ ]:
test_for_single_image('rohit_happy.jpeg', model, labels)

In [ ]:
test_for_single_image('ankita.png', model, labels)